## Setup / Step 0

1. Load some basic libraries and set environment variables for AWS access
2. Initiate tileset to test

In [3]:
%%capture
!pip install -r requirements.txt

In [4]:
import eodc_hub_role
import morecantile
import numpy as np
import pandas as pd
from profiler.main import Timer
import sys
sys.path.append('..')
credentials = eodc_hub_role.fetch_and_set_credentials()

In [5]:
tms = morecantile.tms.get("WebMercatorQuad")
zooms = range(12) # Zoom 10 is the level at which you can see large roads, 15 is buildings
xyz_tiles = []
for z in zooms:
    tile = tms.tile(57, 28, z)
    xyz_tiles.append((tile.x, tile.y, tile.z))

xyz_tiles

[(0, 0, 0),
 (1, 0, 1),
 (2, 1, 2),
 (5, 3, 3),
 (10, 6, 4),
 (21, 13, 5),
 (42, 26, 6),
 (84, 53, 7),
 (168, 107, 8),
 (337, 214, 9),
 (674, 428, 10),
 (1348, 857, 11)]

# Profile titiler-xarray

In [6]:
%load_ext autoreload
%autoreload

import xarray_tile_reader
import zarr_reader
import zarr_helpers

In [7]:
shapes = ['180x360', '180x360', '569x1138', '1799x3599', '5689x11379', 'one_tenth_planet']

In [8]:
variable = 'data'
bucket = 'nasa-eodc-data-store'
datastores = {}
for shape in shapes:
    store_dir = f"s3://{bucket}/fake_data/store_{shape}.zarr"
    ds = zarr_helpers.open_dataset(store_dir)
    ds_specs = zarr_helpers.get_dataset_specs(shape, store_dir, variable, ds)
    datastores[shape] = ds_specs
    nochunk_store_dir = f"s3://{bucket}/fake_data/store_{shape}_nochunks.zarr"
    ds_nochunk = zarr_helpers.open_dataset(nochunk_store_dir)
    key = f"{shape}_nochunks"
    ds_nochunk_specs = zarr_helpers.get_dataset_specs(key, nochunk_store_dir, variable, ds_nochunk)
    datastores[key] = ds_nochunk_specs    

In [9]:
datastores_df = pd.DataFrame.from_dict(datastores, orient='index')
datastores_df.drop(
    columns=['collection_name', 'variable', 'source', 'dtype', 'number_coord_chunks', 'compression']
).sort_values('lat_resolution')

,shape,lat_resolution,lon_resolution,chunk_size_mb,chunks
one_tenth_planet,"{'time': 1, 'lat': 5689, 'lon': 11379}",0.010007,0.010005,1.933838,"{'time': 1, 'lat': 356, 'lon': 712}"
one_tenth_planet_nochunks,"{'time': 1, 'lat': 5689, 'lon': 11379}",0.010007,0.010005,493.889854,"{'time': 1, 'lat': 5689, 'lon': 11379}"
5689x11379,"{'time': 1, 'lat': 5689, 'lon': 11379}",0.031646,0.031640,1.933838,"{'time': 1, 'lat': 356, 'lon': 712}"
5689x11379_nochunks,"{'time': 1, 'lat': 5689, 'lon': 11379}",0.031646,0.031640,493.889854,"{'time': 1, 'lat': 5689, 'lon': 11379}"
1799x3599,"{'time': 1, 'lat': 1799, 'lon': 3599}",0.100111,0.100056,0.772476,"{'time': 1, 'lat': 225, 'lon': 450}"
1799x3599_nochunks,"{'time': 1, 'lat': 1799, 'lon': 3599}",0.100111,0.100056,49.397285,"{'time': 1, 'lat': 1799, 'lon': 3599}"
569x1138,"{'time': 1, 'lat': 569, 'lon': 1138}",0.316901,0.316623,0.310936,"{'time': 1, 'lat': 143, 'lon': 285}"
569x1138_nochunks,"{'time': 1, 'lat': 569, 'lon': 1138}",0.316901,0.316623,4.940201,"{'time': 1, 'lat': 569, 'lon': 1138}"
180x360,"{'time': 1, 'lat': 180, 'lon': 360}",1.005587,1.002786,0.247192,"{'time': 1, 'lat': 90, 'lon': 360}"
180x360_nochunks,"{'time': 1, 'lat': 180, 'lon': 360}",1.005587,1.002786,0.494385,"{'time': 1, 'lat': 180, 'lon': 360}"


## Test different tiles

Below we test the time to generate tiles from each datastore 10 times.

In [10]:
niters = 2
tile_results_df = {}
dataset_chunk_sizes = {}
results_df = datastores.copy()

for xyz_tile in xyz_tiles:
    results_df[xyz_tile] = {}
    for shape, data_specs in datastores.items():
        reference, multiscale = False, False

        results_df[xyz_tile][shape] = {
            'time to open (ms)': [],
            'rio reproject (ms)': [],
            'total time (ms)': []
        }
        timings_results = results_df[xyz_tile][shape]
        for iter in range(niters):
            with Timer() as t:
                image_and_timings, cprofile = xarray_tile_reader.tile(
                    data_specs['source'],
                    *xyz_tile,
                    variable=variable,
                )
            total_time = round(t.elapsed * 1000, 2)

            timings = image_and_timings[1]
            timings_results['time to open (ms)'].append(timings['time_to_open']),
            timings_results['rio reproject (ms)'].append(timings['rio.reproject']),
            timings_results['total time (ms)'].append(total_time)
        timings_results['mean time to open (ms)'] = np.mean(timings_results['time to open (ms)'])
        timings_results['mean rio reproject (ms)'] = np.mean(timings_results['rio reproject (ms)']) 
        timings_results['mean total time (ms)'] = np.mean(timings_results['total time (ms)'])

In [10]:
for xyz_tile in xyz_tiles:
    print(f"Results for {xyz_tile}")
    df = pd.DataFrame.from_dict(results_df[xyz_tile], orient='index')
    df = df.join(datastores_df)
    display(df.sort_values('lat_resolution'))
    print()

Results for (0, 0, 0)


,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
180x360_nochunks,"[215.36, 60.56]","[86.24, 88.68]","[358.82, 205.69]",137.960,87.460,282.255
569x1138,"[59.16, 54.89]","[273.48, 218.24]","[390.94, 333.01]",57.025,245.860,361.975
569x1138_nochunks,"[142.67, 57.94]","[210.4, 98.73]","[410.92, 384.01]",100.305,154.565,397.465
180x360,"[184.12, 61.45]","[349.83, 90.64]","[631.57, 210.41]",122.785,220.235,420.990
1799x3599,"[73.96, 59.13]","[741.89, 557.67]","[875.02, 675.53]",66.545,649.780,775.275
1799x3599_nochunks,"[83.02, 182.39]","[639.03, 639.7]","[779.84, 881.22]",132.705,639.365,830.530
one_tenth_planet,"[107.03, 66.05]","[3802.07, 3019.54]","[3970.21, 3147.51]",86.540,3410.805,3558.860
5689x11379,"[60.49, 104.02]","[3895.01, 3130.64]","[4172.46, 3294.85]",82.255,3512.825,3733.655
one_tenth_planet_nochunks,"[61.8, 66.75]","[10880.84, 6400.21]","[11003.28, 6528.07]",64.275,8640.525,8765.675
5689x11379_nochunks,"[59.0, 54.88]","[13033.0, 6501.24]","[13155.2, 6616.77]",56.940,9767.120,9885.985



Results for (1, 0, 1)


,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
180x360,"[76.5, 94.6]","[74.18, 52.67]","[207.2, 206.53]",85.550,63.425,206.865
569x1138_nochunks,"[146.92, 55.89]","[89.55, 89.48]","[295.89, 204.75]",101.405,89.515,250.320
180x360_nochunks,"[56.29, 71.31]","[50.15, 51.26]","[161.53, 370.91]",63.800,50.705,266.220
569x1138,"[68.75, 104.13]","[199.25, 96.62]","[327.72, 259.82]",86.440,147.935,293.770
1799x3599,"[128.56, 61.55]","[226.8, 187.69]","[415.29, 308.65]",95.055,207.245,361.970
1799x3599_nochunks,"[135.63, 64.29]","[563.51, 553.58]","[757.19, 675.3]",99.960,558.545,716.245
5689x11379,"[92.3, 62.55]","[1006.85, 799.15]","[1158.15, 920.65]",77.425,903.000,1039.400
one_tenth_planet,"[80.29, 67.15]","[2592.37, 2465.82]","[2928.53, 2594.35]",73.720,2529.095,2761.440
5689x11379_nochunks,"[66.69, 67.2]","[5559.37, 5429.34]","[5688.14, 5557.74]",66.945,5494.355,5622.940
one_tenth_planet_nochunks,"[57.33, 55.38]","[5914.81, 5918.84]","[6033.21, 6035.19]",56.355,5916.825,6034.200



Results for (2, 1, 2)


...

,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
180x360_nochunks,"[51.66, 50.94]","[47.42, 47.4]","[154.06, 152.81]",51.300,47.410,153.435
569x1138,"[50.77, 55.18]","[48.26, 47.78]","[154.78, 159.98]",52.975,48.020,157.380
5689x11379,"[56.86, 56.83]","[57.46, 71.92]","[172.36, 185.5]",56.845,64.690,178.930
1799x3599,"[81.28, 57.5]","[54.88, 50.49]","[192.17, 167.26]",69.390,52.685,179.715
180x360,"[90.56, 66.96]","[69.99, 49.74]","[217.01, 174.76]",78.760,59.865,195.885
one_tenth_planet,"[55.18, 59.3]","[104.35, 65.69]","[219.75, 365.01]",57.240,85.020,292.380
569x1138_nochunks,"[58.41, 54.26]","[89.0, 86.97]","[385.32, 199.88]",56.335,87.985,292.600
1799x3599_nochunks,"[53.68, 71.33]","[601.9, 566.85]","[713.74, 698.08]",62.505,584.375,705.910
5689x11379_nochunks,"[57.85, 54.88]","[5292.3, 5305.13]","[5409.45, 5420.57]",56.365,5298.715,5415.010
one_tenth_planet_nochunks,"[71.92, 124.55]","[5360.92, 5342.06]","[5493.59, 5527.41]",98.235,5351.490,5510.500



Results for (674, 428, 10)


,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
180x360,"[57.25, 73.26]","[51.35, 50.36]","[166.86, 180.32]",65.255,50.855,173.590
5689x11379,"[62.94, 56.87]","[58.37, 59.76]","[179.3, 173.98]",59.905,59.065,176.640
569x1138,"[91.18, 61.92]","[50.15, 48.27]","[196.93, 165.31]",76.550,49.210,181.120
180x360_nochunks,"[61.59, 62.09]","[64.5, 62.11]","[183.55, 181.04]",61.840,63.305,182.295
1799x3599,"[101.69, 61.85]","[54.45, 52.7]","[214.18, 169.95]",81.770,53.575,192.065
one_tenth_planet,"[66.89, 63.63]","[94.69, 72.63]","[219.87, 195.82]",65.260,83.660,207.845
569x1138_nochunks,"[55.58, 58.44]","[104.53, 89.43]","[219.01, 387.59]",57.010,96.980,303.300
1799x3599_nochunks,"[64.27, 61.99]","[566.75, 567.0]","[689.56, 688.66]",63.130,566.875,689.110
5689x11379_nochunks,"[61.19, 58.55]","[5305.8, 5304.53]","[5426.0, 5423.17]",59.870,5305.165,5424.585
one_tenth_planet_nochunks,"[63.67, 60.79]","[5263.28, 5442.13]","[5388.16, 5562.6]",62.230,5352.705,5475.380



Results for (1348, 857, 11)


,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
569x1138,"[64.48, 61.74]","[49.63, 52.43]","[170.34, 171.93]",63.110,51.030,171.135
180x360_nochunks,"[61.48, 60.72]","[50.1, 50.9]","[176.59, 174.57]",61.100,50.500,175.580
180x360,"[58.95, 63.29]","[51.01, 50.12]","[176.14, 176.67]",61.120,50.565,176.405
5689x11379,"[70.98, 62.84]","[60.58, 56.47]","[189.76, 175.39]",66.910,58.525,182.575
569x1138_nochunks,"[59.38, 55.88]","[87.62, 86.22]","[207.31, 199.99]",57.630,86.920,203.650
one_tenth_planet,"[60.66, 63.92]","[157.35, 98.78]","[274.98, 221.86]",62.290,128.065,248.420
1799x3599,"[66.68, 64.77]","[59.78, 53.53]","[184.38, 358.83]",65.725,56.655,271.605
1799x3599_nochunks,"[106.99, 65.12]","[562.53, 561.5]","[730.51, 686.64]",86.055,562.015,708.575
5689x11379_nochunks,"[64.25, 60.49]","[5260.48, 5259.0]","[5384.1, 5378.7]",62.370,5259.740,5381.400
one_tenth_planet_nochunks,"[65.24, 87.16]","[5312.51, 5316.81]","[5437.23, 5465.33]",76.200,5314.660,5451.280


In [32]:
# Dimensions of the grid
grid_width = 5689
grid_height = 11379

# Earth's circumference at the equator in kilometers
earth_circumference_km = 40075

# Calculate the extent of coverage area in kilometers
extent_width_km = earth_circumference_km / grid_width
extent_height_km = earth_circumference_km / grid_height

print(f"Resolution of the global dataset: {extent_width_km} km x {extent_height_km} km")

Resolution of the global dataset: 7.044296009843558 km x 3.5218384743826348 km
